##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar o mecanismo de tolerância a falhas

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/fault_tolerance"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/fault_tolerance.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/fault_tolerance.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/fault_tolerance.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Tolerância a falhas refere-se a um mecanismo de salvar periodicamente os estados de objetos rastreáveis, como parâmetros e modelos. Isto permite recuperá-los em caso de falha do programa/máquina durante o treinamento.

Este guia primeiro demonstra como incluir tolerância a falhas no treinamento com `tf.estimator.Estimator` no TensorFlow 1 especificando o salvamento de métricas com `tf.estimator.RunConfig`. Em seguida, você aprenderá como implementar a tolerância a falhas para treinamentos no Tensorflow 2 de duas maneiras:

- Se você usar a API Keras `Model.fit`, poderá passar o callback `tf.keras.callbacks.BackupAndRestore` para ela.
- Se você usar um loop de treinamento personalizado (com `tf.GradientTape`), poderá salvar checkpoints arbitrariamente usando as APIs `tf.train.Checkpoint` e `tf.train.CheckpointManager`.

Ambas as soluções farão backup e restaurarão os estados de treinamento nos arquivos de [checkpoint](../../guide/checkpoint.ipynb).


## Configuração

Instale `tf-nightly`, pois a frequência de salvamento de checkpoints em um determinado passo com o argumento `save_freq` em `tf.keras.callbacks.BackupAndRestore` foi lançada no TensorFlow 2.10:

In [ ]:
!pip install tf-nightly

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import numpy as np
import tempfile
import time

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## TensorFlow 1: salve checkpoints com `tf.estimator.RunConfig`

No TensorFlow 1, você pode configurar um `tf.estimator` para salvar checkpoints a cada passo configurando `tf.estimator.RunConfig`.

Neste exemplo, comece escrevendo um hook que lança artificialmente um erro durante o quinto checkpoint:

In [ ]:
class InterruptHook(tf1.train.SessionRunHook):
  # A hook for artificially interrupting training.
  def begin(self):
    self._step = -1

  def before_run(self, run_context):
    self._step += 1

  def after_run(self, run_context, run_values):
    if self._step == 5:
      raise RuntimeError('Interruption')

Em seguida, configure `tf.estimator.Estimator` para salvar cada checkpoint e usar o dataset MNIST:

In [ ]:
feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]
config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.2,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

Comece a treinar o modelo. Uma exceção artificial será levantada pelo hook que você definiu anteriormente.

In [ ]:
try:
  classifier.train(input_fn=train_input_fn,
                   hooks=[InterruptHook()],
                   max_steps=10)
except Exception as e:
  print(f'{type(e).__name__}:{e}')

Recrie o `tf.estimator.Estimator` usando o último checkpoint salvo e continue treinando:

In [ ]:
classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.2,
    model_dir=path,
    config = config
)
classifier.train(input_fn=train_input_fn,
                   max_steps = 10)

## TensorFlow 2: faça backup e restauração com um callback e `Model.fit`

No TensorFlow 2, se você usar a API Keras `Model.fit` para treinamento, poderá fornecer o callback `tf.keras.callbacks.BackupAndRestore` para incluir a funcionalidade de tolerância a falhas.

Para ajudar a demonstrar isto, primeiro comece definindo uma classe Keras `Callback` que lança artificialmente um erro durante o quarto checkpoint da época:


In [ ]:
class InterruptAtEpoch(tf.keras.callbacks.Callback):
  # A callback for artificially interrupting training.
  def __init__(self, interrupting_epoch=3):
    self.interrupting_epoch = interrupting_epoch

  def on_epoch_end(self, epoch, log=None):
    if epoch == self.interrupting_epoch:
      raise RuntimeError('Interruption')

Em seguida, defina e instancie um modelo Keras simples, defina a função de perda, chame `Model.compile` e configure um callback `tf.keras.callbacks.BackupAndRestore` que salvará os checkpoints num diretório temporário nos limites da época:

In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
  ])
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model = create_model()
model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'])
log_dir = tempfile.mkdtemp()
backup_restore_callback = tf.keras.callbacks.BackupAndRestore(
    backup_dir = log_dir)

Comece a treinar o modelo com `Model.fit`. Durante o treinamento, os checkpoints serão salvos graças a `tf.keras.callbacks.BackupAndRestore` instanciado acima, enquanto a classe `InterruptAtEpoch` gerará uma exceção artificial para simular uma falha depois da quarta época.

In [ ]:
try:
  model.fit(x=x_train,
            y=y_train,
            epochs=10,
            steps_per_epoch=100,
            validation_data=(x_test, y_test),
            callbacks=[backup_restore_callback, InterruptAtEpoch()])
except Exception as e:
  print(f'{type(e).__name__}:{e}')

Em seguida, instancie o modelo Keras, chame `Model.compile` e continue treinando o modelo com `Model.fit` a partir de um checkpoint salvo anteriormente:

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'],
              steps_per_execution=10)
model.fit(x=x_train,
            y=y_train,
            epochs=10,
            steps_per_epoch=100,
            validation_data=(x_test, y_test),
            callbacks=[backup_restore_callback])

Defina outra classe `Callback` que lance artificialmente um erro durante o 140º passo:

In [ ]:
class InterruptAtStep(tf.keras.callbacks.Callback):
  # A callback for artificially interrupting training.
  def __init__(self, interrupting_step=140):
    self.total_step_count = 0
    self.interrupting_step = interrupting_step

  def on_batch_begin(self, batch, logs=None):
    self.total_step_count += 1

  def on_batch_end(self, batch, logs=None):
    if self.total_step_count == self.interrupting_step:
      print("\nInterrupting at step count", self.total_step_count)
      raise RuntimeError('Interruption')

Observação: esta seção usa recursos que estão disponíveis apenas na versão `tf-nightly` até o lançamento do Tensorflow 2.10.

Para garantir que os checkpoints sejam salvos a cada 30 etapas, defina `save_freq` no callback `BackupAndRestore` como `30`. O `InterruptAtStep` gerará uma exceção artificial para simular uma falha na época 1 e no passo 40 (contagem total de passos 140). O checkpoint seria salvo pela última vez na época 1 e no passo 20.

In [ ]:
log_dir_2 = tempfile.mkdtemp()

backup_restore_callback = tf.keras.callbacks.BackupAndRestore(
    backup_dir = log_dir_2, save_freq=30
)
model = create_model()
model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'])
try:
  model.fit(x=x_train,
            y=y_train,
            epochs=10,
            steps_per_epoch=100,
            validation_data=(x_test, y_test),
            callbacks=[backup_restore_callback, InterruptAtStep()])
except Exception as e:
  print(f'{type(e).__name__}:{e}')

Em seguida, instancie o modelo Keras, chame `Model.compile` e continue treinando o modelo com `Model.fit` a partir de um checkpoint salvo anteriormente. Observe que o treinamento começa na época 2 e no passo 21.

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'],
              steps_per_execution=10)
model.fit(x=x_train,
            y=y_train,
            epochs=10,
            steps_per_epoch=100,
            validation_data=(x_test, y_test),
            callbacks=[backup_restore_callback])

## TensorFlow 2: escreva checkpoints manuais com um loop de treinamento personalizado

Se você usar um loop de treinamento personalizado no TensorFlow 2, poderá implementar um mecanismo de tolerância a falhas com as APIs `tf.train.Checkpoint` e `tf.train.CheckpointManager`.

Este exemplo demonstra como:

- Usar um objeto `tf.train.Checkpoint` para criar um checkpoint manualmente, onde os objetos rastreáveis ​​que você deseja salvar são definidos como atributos.
- Usar um `tf.train.CheckpointManager` para gerenciar múltiplos checkpoints.

Comece definindo e instanciando o modelo Keras, o otimizador e a função de perda. Em seguida, crie um `Checkpoint` que gerencie dois objetos com estados rastreáveis ​​(o modelo e o otimizador), bem como um `CheckpointManager` para registrar e manter diversos checkpoints num diretório temporário.

In [ ]:
model = create_model()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
log_dir = tempfile.mkdtemp()
epochs = 5
steps_per_epoch = 5

checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint_manager = tf.train.CheckpointManager(
            checkpoint, log_dir, max_to_keep=2)

Agora, implemente um loop de treinamento personalizado onde, depois da primeira época, toda vez que uma nova época iniciar, o último checkpoint seja carregado:

In [ ]:
for epoch in range(epochs):
  if epoch > 0:
      tf.train.load_checkpoint(save_path)
  print(f"\nStart of epoch {epoch}")

  for step in range(steps_per_epoch):
    with tf.GradientTape() as tape:

      logits = model(x_train, training=True)
      loss_value = loss_fn(y_train, logits)

      grads = tape.gradient(loss_value, model.trainable_weights)
      optimizer.apply_gradients(zip(grads, model.trainable_weights))

    save_path = checkpoint_manager.save()
    print(f"Checkpoint saved to {save_path}")
    print(f"Training loss at step {step}: {loss_value}")

## Próximos passos

Para saber mais sobre tolerância a falhas e checkpoints no TensorFlow 2, veja a seguinte documentação:

- A documentação da API de callbacks `tf.keras.callbacks.BackupAndRestore`.
- A documentação das APIs `tf.train.Checkpoint` e `tf.train.CheckpointManager`.
- O guia [Checkpoints de treinamento](../../guide/checkpoint.ipynb), incluindo a seção *Escrevendo checkpoints* .

Você também pode achar útil o seguinte material relacionado ao [treinamento distribuído](../..guide/distributed_training.ipynb):

- A seção *Tolerância a falhas* no tutorial [Treinamento multifuncional com Keras](../../tutorials/distribute/multi_worker_with_keras.ipynb).
- A seção *Lidando com falhas em tarefas* no tutorial [Treinamento do servidor de parâmetros](../../tutorials/distribute/parameter_server_training.ipynb).